In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
housing = fetch_california_housing()

In [5]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [8]:
housing

{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
           37.88      , -122.23      ],
        [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
           37.86      , -122.22      ],
        [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
           37.85      , -122.24      ],
        ...,
        [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
           39.43      , -121.22      ],
        [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
           39.43      , -121.32      ],
        [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
           39.37      , -121.24      ]]),
 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894]),
 'frame': None,
 'target_names': ['MedHouseVal'],
 'feature_names': ['MedInc',
  'HouseAge',
  'AveRooms',
  'AveBedrms',
  'Population',
  'AveOccup',
  'Latitude',
  'Longitude'],
 'DESCR': '.. _california_housing_dataset:\n

In [5]:
scaler = StandardScaler()

In [6]:
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [7]:
from scipy.stats import reciprocal 

In [8]:
import numpy as np

In [9]:
param_distribs = {
    "model__n_hidden": [0, 1, 2, 3],
    "model__n_neurons": np.arange(1, 100),
    "model__learning_rate": reciprocal(3e-4, 3e-2).rvs(1000).tolist(),
    "model__optimizer": ["adam", "sgd", "nesterov"]
}

In [10]:
from sklearn.model_selection import RandomizedSearchCV

In [11]:
import tensorflow as tf

In [12]:
import scikeras
from scikeras.wrappers import KerasRegressor 

In [13]:
def build_model(n_hidden, n_neurons, optimizer, learning_rate):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=X_train[0].shape))
    
    for layer in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(1))
    
    if optimizer == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "nesterov":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, nesterov=True)
        
    model.compile(loss="mse", optimizer=optimizer, metrics=["mse"])
    return model

In [14]:
es = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=1.0, verbose=1)

In [15]:
keras_reg = KerasRegressor(build_model, callbacks=[es])

In [16]:
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=5, cv=3, verbose=2)

rnd_search_cv.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid), verbose=0)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 23: early stopping
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 268us/step
[CV] END model__learning_rate=0.0007625817608721506, model__n_hidden=0, model__n_neurons=41, model__optimizer=adam; total time=   2.7s


/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 31: early stopping
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 267us/step
[CV] END model__learning_rate=0.0007625817608721506, model__n_hidden=0, model__n_neurons=41, model__optimizer=adam; total time=   3.5s


...

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 11: early stopping
121/121 ━━━━━━━━━━━━━━━━━━━━ 0s 327us/step
[CV] END model__learning_rate=0.004370296752215607, model__n_hidden=2, model__n_neurons=38, model__optimizer=sgd; total time=   1.5s


/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/keras/src/layers/core/input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 11: early stopping


RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(callbacks=[<keras.src.callbacks.early_stopping.EarlyStopping object at 0x146a5cd60>], model=<function build_model at 0x15686a940>),
                   n_iter=5,
                   param_distributions={'model__learning_rate': [0.0005551478739698822,
                                                                 0.0006591440753654614,
                                                                 0.008047004167315824,
                                                                 0.002131577748970913,
                                                                 0.012039971345849032,
                                                                 0.00080676339556...
                                        'model__n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
                                        'model__optimizer': ['adam', 'sgd',
                                                             'nesterov']},
                   verbose=2)

In [17]:
rnd_search_cv.best_params_

{'model__optimizer': 'adam',
 'model__n_neurons': 53,
 'model__n_hidden': 2,
 'model__learning_rate': 0.0020062168993894075}

In [18]:
import pickle

In [19]:
with open('rnd_search_params.pkl', 'wb') as f:
    pickle.dump(rnd_search_cv.best_params_, f)

In [20]:
with open('rnd_search_scikeras.pkl', 'wb') as f:
    pickle.dump(rnd_search_cv, f)

In [21]:
import keras_tuner as kt

In [41]:
def build_model(hp):
    n_hidden = hp.Int('n_hidden', min_value=0, max_value=3, default=2)
    n_neurons = hp.Int('n_neurons', min_value=1, max_value=100, default=30)
    learning_rate = hp.Choice('learning_rate', values=[1e-3, 3e-3, 1e-2, 3e-2])
    optimizer = hp.Choice('optimizer', values=['adam', 'sgd', 'nesterov'])

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=X_train[0].shape))

    for layer in range(n_hidden):
        model.add(tf.keras.layers.Dense(n_neurons, activation="relu"))
    model.add(tf.keras.layers.Dense(1))

    if optimizer == "adam":
        optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == "sgd":
        optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "nesterov":
        optimizer = tf.keras.optimizers.NAG(learning_rate=learning_rate)

    model.compile(loss="mse", optimizer=optimizer, metrics=["mse"])
    return model

In [42]:
random_search_tuner = kt.RandomSearch(build_model, objective="val_mse", max_trials=10, overwrite=True, directory="my_california_housing", project_name="my_rnd_search")

In [43]:
import os

In [44]:
root_logdir = os.path.join(random_search_tuner.project_dir, 'tensorboard')
tb = tf.keras.callbacks.TensorBoard(root_logdir)

In [45]:
random_search_tuner.search(X_train, y_train, callbacks=[tb, es], validation_data=(X_valid, y_valid), epochs=100)

Trial 10 Complete [00h 00m 00s]

Best val_mse So Far: 0.2776722311973572
Total elapsed time: 00h 00m 19s


In [46]:
random_search_tuner.results_summary()

Results summary
Results in my_california_housing/my_rnd_search
Showing 10 best trials
Objective(name="val_mse", direction="min")

Trial 07 summary
Hyperparameters:
n_hidden: 3
n_neurons: 52
learning_rate: 0.003
optimizer: adam
Score: 0.2776722311973572

Trial 06 summary
Hyperparameters:
n_hidden: 2
n_neurons: 53
learning_rate: 0.003
optimizer: adam
Score: 0.30790621042251587

Trial 05 summary
Hyperparameters:
n_hidden: 2
n_neurons: 65
learning_rate: 0.03
optimizer: sgd
Score: 0.31408748030662537

Trial 04 summary
Hyperparameters:
n_hidden: 2
n_neurons: 95
learning_rate: 0.01
optimizer: sgd
Score: 0.33419594168663025

Trial 02 summary
Hyperparameters:
n_hidden: 1
n_neurons: 16
learning_rate: 0.001
optimizer: sgd
Score: 0.4622598886489868

Trial 08 summary
Hyperparameters:
n_hidden: 0
n_neurons: 47
learning_rate: 0.03
optimizer: sgd
Score: 0.589679479598999

Trial 00 summary
Hyperparameters:
n_hidden: 3
n_neurons: 82
learning_rate: 0.003
optimizer: nesterov
Traceback (most recent call la

In [47]:
random_search_tuner.get_best_hyperparameters()[0].values

{'n_hidden': 3, 'n_neurons': 52, 'learning_rate': 0.003, 'optimizer': 'adam'}

In [48]:
with open('kt_search_params.pkl', 'wb') as f:
    pickle.dump(random_search_tuner.get_best_hyperparameters()[0].values, f)

In [49]:
random_search_tuner.get_best_models(num_models=1)[0].save('kt_best_model.keras')

/opt/homebrew/Caskroom/miniforge/base/envs/ml/lib/python3.9/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [50]:
%load_ext tensorboard
%tensorboard --logdir {root_logdir}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 83517), started 1:02:20 ago. (Use '!kill 83517' to kill it.)